# Modifying the `Schedule` object

GeNet has a number of methods to change the Schedule objects. Make sure you validate the end result.

In [1]:
# read example network
import pandas as pd
from genet import Network, Stop, Route, Service, read_matsim
import os

path_to_matsim_network = '../example_data/pt2matsim_network'

network = os.path.join(path_to_matsim_network, 'network.xml')
schedule = os.path.join(path_to_matsim_network, 'schedule.xml')
vehicles = os.path.join(path_to_matsim_network, 'vehicles.xml')
n = read_matsim(
    path_to_network=network, 
    epsg='epsg:27700', 
    path_to_schedule=schedule, 
    path_to_vehicles=vehicles
)
# you don't need to read the vehicles file, but doing so ensures all vehicles
# in the schedule are of the expected type and the definition of the vehicle
# is preserved
n.print()

Graph info: Name: Network graph
Type: MultiDiGraph
Number of nodes: 1662
Number of edges: 3166
Average in degree:   1.9049
Average out degree:   1.9049 
Schedule info: Schedule:
Number of services: 9
Number of routes: 68
Number of stops: 118


## Adding Routes, Services

You can add `Route`s and `Service`s. To add a `Route`, you need to identify which existing `Service` it should come under.

In [2]:
route = Route(
    route_short_name='N55',
    mode='bus',
    trips={'trip_id': ['fun_trip_1', 'fun_trip_2'], 
           'trip_departure_time': ['03:53:00', '16:23:00'], 
           'vehicle_id': ['fun_bus_1', 'fun_bus_2']},
   arrival_offsets=['00:00:00', '00:02:00', '00:04:00', '00:06:00'],
   departure_offsets=['00:00:00', '00:02:00', '00:04:00', '00:06:00'],
   id='new_route',
#    route= ['834', '1573', '3139', '3141', '574', '3154', '979', '980', '981'],
   await_departure= [True, True, True, True],
   stops=[n.schedule.stop('490000235X'),
          Stop(id='new_stop', x=529500, y=181300,
            name='New Stop', epsg='epsg:27700'),
          Stop(id='other_new_stop', x=529502, y=181302,
            name='Other New Stop', epsg='epsg:27700'),
          n.schedule.stop('490010689KB')]
)

You can create and add a new `Service`, or add the route to an existing Service

In [3]:
new_service = Service(id='new_service', routes=[route])
new_service.print()

Service ID: new_service
Name: N55
Number of routes: 1
Number of stops: 4


In [4]:
n.schedule.add_service(Service(id='new_service', routes=[route]))

2021-10-25 11:28:45,538 - Added Service with index `new_service`, data={'id': 'new_service', 'name': 'N55'} and Routes: ['new_route']


<Service instance at 4464896976: with 1 routes>

In [5]:
n.schedule.add_route('20274', route)

2021-10-25 11:28:46,113 - Route with ID `new_route` within already exists in the Schedule. This Route will be reindexed to `20274_4`
2021-10-25 11:28:46,126 - Reindexed Route from new_route to 20274_4
2021-10-25 11:28:46,138 - Added Route with index `20274_4`, data={'route_short_name': 'N55', 'mode': 'bus', 'trips': {'trip_id': ['fun_trip_1', 'fun_trip_2'], 'trip_departure_time': ['03:53:00', '16:23:00'], 'vehicle_id': ['fun_bus_1', 'fun_bus_2']}, 'arrival_offsets': ['00:00:00', '00:02:00', '00:04:00', '00:06:00'], 'departure_offsets': ['00:00:00', '00:02:00', '00:04:00', '00:06:00'], 'route_long_name': '', 'id': '20274_4', 'route': [], 'await_departure': [True, True, True, True], 'ordered_stops': ['490000235X', 'new_stop', 'other_new_stop', '490010689KB']} to Service `20274` within the Schedule
/Users/kasia.kozlowska/PycharmProjects/ABM/genet/genet/schedule_elements.py:1243: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  self.vehicles = {**df.T.to_dict(

<Route instance at 4910081296: with 4 stops and 2 trips>

In [6]:
n.schedule['new_service'].print()

Service ID: new_service
Name: N55
Number of routes: 1
Number of stops: 4


In [7]:
n.schedule.route('20274_4').ordered_stops

['490000235X', 'new_stop', 'other_new_stop', '490010689KB']

You can also refer to existing stops in the `Schedule` when creating a`Route` to be added. You can either just pass Stop IDs as strings or use a stop method on the schedule to take and use that stop object. Note that in the case of the former (passing ID strings), the route will not have the spatial information for those stops until it is added to the Schedule.

In [8]:
r = Route(
    route_short_name='N55',
    mode='bus',
    trips={'trip_id': ['some_trip_1'], 
           'trip_departure_time': ['16:23:00'], 
           'vehicle_id': ['some_bus_2']},
   arrival_offsets=['00:00:00', '00:06:00'],
   departure_offsets=['00:00:00', '00:06:00'],
   id='another_new_route',
#    route= ['834', '1573', '3139', '3141', '574', '3154', '979', '980', '981'],
   await_departure= [True, True],
   stops=['490000235X.link:834', 
          '490010689KB.link:981']
)

In [9]:
r.graph().nodes['490000235X.link:834']

{'routes': {'another_new_route'}}

In [10]:
n.schedule.add_route('20274', r)

2021-10-25 11:28:46,953 - Added Route with index `another_new_route`, data={'route_short_name': 'N55', 'mode': 'bus', 'trips': {'trip_id': ['some_trip_1'], 'trip_departure_time': ['16:23:00'], 'vehicle_id': ['some_bus_2']}, 'arrival_offsets': ['00:00:00', '00:06:00'], 'departure_offsets': ['00:00:00', '00:06:00'], 'route_long_name': '', 'id': 'another_new_route', 'route': [], 'await_departure': [True, True], 'ordered_stops': ['490000235X.link:834', '490010689KB.link:981']} to Service `20274` within the Schedule


<Route instance at 4915961552: with 2 stops and 1 trips>

In [11]:
r.graph().nodes['490000235X.link:834']

{'services': {'14134', '18853', '20274'},
 'routes': {'VJ12ba6089dfb2733e29c415a1a0015fef30fd5305',
  'VJ256e98df611ff48afe737ddc81cbcde82e4e81c8',
  'VJ2aba67e3ed98f2ed5f5966c1ac394cbf6d1943d7',
  'VJ375a660d47a2aa570aa20a8568012da8497ffecf',
  'VJ4e2b897edf0e7b8a8e3b5516ab43ce56f72c5cff',
  'VJa7f37392e276aeac26c7e73bbc05e6a71af38dba',
  'VJd78967364a302cf232c5139d40622dcb6c238c9e',
  'VJdf3936da1a51eb33db594ef99738802c14b19995',
  'VJf3e316e5e605bb512147dee2a989be5a82ef1b5f',
  'VJf9a22035ae6f25bb420df833474943ad76065c89',
  'another_new_route'},
 'id': '490000235X.link:834',
 'x': 529981.7958802709,
 'y': 181412.0975758662,
 'epsg': 'epsg:27700',
 'name': 'Tottenham Court Road Station (Stop X)',
 'lon': -0.12809598708996447,
 'lat': 51.51668503324075,
 's2_id': 5221390722025467597,
 'linkRefId': '834',
 'isBlocking': 'false'}

In [12]:
r = Route(
    route_short_name='N55',
    mode='bus',
    trips={'trip_id': ['some_trip_1'], 
           'trip_departure_time': ['16:23:00'], 
           'vehicle_id': ['some_bus_2']},
   arrival_offsets=['00:00:00', '00:06:00'],
   departure_offsets=['00:00:00', '00:06:00'],
   id='another_new_route_2',
#    route= ['834', '1573', '3139', '3141', '574', '3154', '979', '980', '981'],
   await_departure= [True, True],
   stops=[n.schedule.stop('490000235X.link:834'), 
          n.schedule.stop('490010689KB.link:981')]
)
n.schedule.add_route('20274', r)

2021-10-25 11:28:47,948 - Added Route with index `another_new_route_2`, data={'route_short_name': 'N55', 'mode': 'bus', 'trips': {'trip_id': ['some_trip_1'], 'trip_departure_time': ['16:23:00'], 'vehicle_id': ['some_bus_2']}, 'arrival_offsets': ['00:00:00', '00:06:00'], 'departure_offsets': ['00:00:00', '00:06:00'], 'route_long_name': '', 'id': 'another_new_route_2', 'route': [], 'await_departure': [True, True], 'ordered_stops': ['490000235X.link:834', '490010689KB.link:981']} to Service `20274` within the Schedule


<Route instance at 4912081168: with 2 stops and 1 trips>

Note that for a Schedule to be a valid MATSim network, each stop referred to by a route needs a `linkRefId` attribute which links the stop to the `Network`.

Trying to add Stops with IDs already in the Schedule will result in an error, unless the `force=True` is set. The added route/service will inherit the data stored under those stops. The idea is that you can either specify the Stops in the route/service to be added correctly, or if they are to be changed, you use a dedicated method - check section 'Modifying data' below.

In [13]:
route = Route(
    route_short_name='N55',
    mode='bus',
    trips={'trip_id': ['fun_trip_1', 'fun_trip_2'], 
           'trip_departure_time': ['03:53:00', '16:23:00'], 
           'vehicle_id': ['fun_bus_1', 'fun_bus_2']},
   arrival_offsets=['00:00:00', '00:02:00', '00:04:00', '00:06:00'],
   departure_offsets=['00:00:00', '00:02:00', '00:04:00', '00:06:00'],
   id='another_new_route_3',
#    route= ['834', '1573', '3139', '3141', '574', '3154', '979', '980', '981'],
   await_departure= [True, True, True, True],
   stops=[
       Stop(id='490000235X.link:834', x=529981, y=181412, epsg='epsg:27700'),
       Stop(id='new_stop', x=529500, y=181300, epsg='epsg:27700', name='New Stop'),
       Stop(id='other_new_stop', x=529502, y=181302, epsg='epsg:27700', name='Other New Stop'),
       Stop(id='490010689KB.link:981', x=529166, y=181256, epsg='epsg:27700')
   ]
)

n.schedule.add_route('20274', route, force=True)

2021-10-25 11:28:50,986 - The following stops will inherit the data currently stored under those Stop IDs in the Schedule: ['490000235X.link:834', '490010689KB.link:981'].
2021-10-25 11:28:50,994 - Added Route with index `another_new_route_3`, data={'route_short_name': 'N55', 'mode': 'bus', 'trips': {'trip_id': ['fun_trip_1', 'fun_trip_2'], 'trip_departure_time': ['03:53:00', '16:23:00'], 'vehicle_id': ['fun_bus_1', 'fun_bus_2']}, 'arrival_offsets': ['00:00:00', '00:02:00', '00:04:00', '00:06:00'], 'departure_offsets': ['00:00:00', '00:02:00', '00:04:00', '00:06:00'], 'route_long_name': '', 'id': 'another_new_route_3', 'route': [], 'await_departure': [True, True, True, True], 'ordered_stops': ['490000235X.link:834', 'new_stop', 'other_new_stop', '490010689KB.link:981']} to Service `20274` within the Schedule


<Route instance at 4912020304: with 4 stops and 2 trips>

Note the message above: `The following stops will inherit the data currently stored under those Stop IDs in the Schedule: ['490000235X.link:834', '490010689KB.link:981'].`

NOTE: adding routes and services results in new vehicles (unless you reuse the ones already in the Schedule---beware that the same vehicle cannot service multiple trips at the same time, genet does not currently have checks for this, the user needs to be mindful of the physics of shared vehicles). New vehicles need definitions, you can add them yourself to `schedule.vehicles['vehicle_id'] = {'type': 'bus'}` ensuring this vehicle type is defined in `schedule.vehicle_types['bus']`, or you can use a genet method to generate those vehicles, the type will be derived from the mode of the route. Then you can check if all of the types that vehicles are referring to have definitions.

In [14]:
len(n.schedule.vehicles)

13293

In [15]:
n.schedule.generate_vehicles()

In [16]:
n.schedule.validate_vehicle_definitions()

2021-10-25 11:28:52,108 - The following vehicle types are missing from the `vehicle_types` attribute: {'bus'}
2021-10-25 11:28:52,115 - Vehicles affected by missing vehicle types: {'fun_bus_1': {'type': 'bus'}, 'fun_bus_2': {'type': 'bus'}, 'some_bus_2': {'type': 'bus'}}


False

In [17]:
len(n.schedule.vehicles)

13293

In [18]:
n.schedule.change_log().tail()

timestamp change_event object_type old_id               new_id  \
0  2021-10-25 11:28:45          add     service   None          new_service   
1  2021-10-25 11:28:46          add       route   None              20274_4   
2  2021-10-25 11:28:46          add       route   None    another_new_route   
3  2021-10-25 11:28:47          add       route   None  another_new_route_2   
4  2021-10-25 11:28:50          add       route   None  another_new_route_3   

  old_attributes                                     new_attributes  \
0           None               {'id': 'new_service', 'name': 'N55'}   
1           None  {'route_short_name': 'N55', 'mode': 'bus', 'tr...   
2           None  {'route_short_name': 'N55', 'mode': 'bus', 'tr...   
3           None  {'route_short_name': 'N55', 'mode': 'bus', 'tr...   
4           None  {'route_short_name': 'N55', 'mode': 'bus', 'tr...   

                                                diff  
0  [(add, , [('id', 'new_service'), ('name', 'N55...  
1  [(add, , [('route_short_name', 'N55'), ('mode'...  
2  [(add, , [('route_short_name', 'N55'), ('mode'...  
3  [(add, , [('route_short_name', 'N55'), ('mode'...  
4  [(add, , [('route_short_name', 'N55'), ('mode'...

There are no methods to add `Stop` objects on their own. They are added to the Schedule with `Route` and `Service` objects.

### Routing added Services/Routes

You can use methods in GeNet to relate the Stops of PT services and find network routes between them. First you need to know the ID of the Service you want to snap (you can also snap the entire schedule, but depending on the size and complexity of your network and schedule it might take a long time). GeNet will then relate all stops in that service to a link in the Network graph and route all of the `Route`s of the `Service` object. It will do this for directed subsets (subgraphs) of the Service (e.g. if you have a Northbound and Southboud service, the stops may have to find different links in the graph)

There are a lot of different parameters you can add to this method, that give you a bit more control. 
- `solver`: You can specify different mathematical solvers. For example GLPK, an open source solver which can be found here: https://www.gnu.org/software/glpk/. Another good open source choice is CBC: https://projects.coin-or.org/Cbc. You specify it as a string e.g. 'glpk', 'cbc', 'gurobi'. The solver needs to support MILP - mixed integer linear programming
- `allow_partial`: Sometimes there isn't a link available for snapping within threshold for all stops. If allowed (default) an artificial self-loop link will be created as well as any connecting links to that unsnapped stop, under modal conditions. If set to False and the problem is partial, it will raise PartialMaxStableSetProblem error instead.
- `distance_threshold`, `step_size`: You can vary the threshold for snapping stops to links. There are two parameters, the overall threshold and a step size; when snapping, the search area for links increases in steps until some links are found, we don't use the threshold right away because we don't want to have too many choices for snapping (makes it a little less heavy computationally)).
- `additional_modes`: You can specify additional modes (when snapping, genet will consider a modal subset of the network that matches the pt service, e.g. bus, but you might want to use links that allow cars too).
- `allow_directional_split`: You can opt for splitting the problem of snapping by direction. GeNet will then solve a series of problems for subgraphs (disjoint w.r.t. edges) of the Service. This is useful for rail type services that might share the same stop regardless of direction (in comparison to buses which have distinct bus stops, depending on which way they're travelling).

Check the methods' doc strings for up to date details.

In [19]:
n.schedule['new_service'].route('new_route').route

[]

In [20]:
n.schedule['new_service'].route('new_route').ordered_stops

['490000235X', 'new_stop', 'other_new_stop', '490010689KB']

In [21]:
n.route_service('new_service')

2021-10-25 11:28:55,733 - Routing Service new_service with modes = {'bus'}
2021-10-25 11:28:55,846 - Building Maximum Stable Set for PT graph with 4 stops and 3 edges
2021-10-25 11:28:58,420 - This Maximum Stable Set Problem is partially viable.
2021-10-25 11:28:58,421 - Maximum Stable Set problem to snap the PT graph to the network is partially viable, meaning not all stops have found a link to snap to within the distance_threshold.Partial snapping is ON, this problem will proceed to the solver.
2021-10-25 11:28:58,424 - Passing problem to solver
2021-10-25 11:28:58,428 - Initializing ordered Set vertices with a fundamentally unordered data source (type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
2021-10-25 11:28:58,436 - Passing problem to solver
2021-10-25 11:28:58,930 - Successfully snapped 3 stops to network links.
2021-10-25 11:28:59,034 - Stop ID changes detected for Routes: {'new_route'}
2021-10-25 11:28:59,038 - Changed Route attributes for 1 route

In [22]:
n.schedule['new_service'].route('new_route').route

['artificial_link===from:490000235X===to:490000235X',
 'artificial_link===from:490000235X===to:9521035',
 '3154',
 '979',
 '980',
 '981']

In [23]:
n.schedule['new_service'].route('new_route').ordered_stops

['490000235X.link:artificial_link===from:490000235X===to:490000235X',
 'new_stop.link:3154',
 'other_new_stop.link:3154',
 '490010689KB.link:981']

## Reindexing

In [24]:
n.schedule['new_service'].reindex(new_id='more_appropriate_id')

2021-10-25 11:28:59,561 - Reindexed Service from new_service to more_appropriate_id


In [25]:
n.schedule.route('new_route').reindex(new_id='more_appropriate_route_id')

2021-10-25 11:28:59,584 - Reindexed Route from new_route to more_appropriate_route_id


In [26]:
n.schedule.change_log().tail()

timestamp change_event object_type       old_id  \
3  2021-10-25 11:28:47          add       route         None   
4  2021-10-25 11:28:50          add       route         None   
5  2021-10-25 11:28:59       modify       route    new_route   
6  2021-10-25 11:28:59       modify     service  new_service   
7  2021-10-25 11:28:59       modify       route    new_route   

                      new_id  \
3        another_new_route_2   
4        another_new_route_3   
5                  new_route   
6        more_appropriate_id   
7  more_appropriate_route_id   

                                      old_attributes  \
3                                               None   
4                                               None   
5  {'route_short_name': 'N55', 'mode': 'bus', 'tr...   
6                              {'id': 'new_service'}   
7                                {'id': 'new_route'}   

                                      new_attributes  \
3  {'route_short_name': 'N55', 'mode': 'bus', 'tr...   
4  {'route_short_name': 'N55', 'mode': 'bus', 'tr...   
5  {'route_short_name': 'N55', 'mode': 'bus', 'tr...   
6                      {'id': 'more_appropriate_id'}   
7                {'id': 'more_appropriate_route_id'}   

                                                diff  
3  [(add, , [('route_short_name', 'N55'), ('mode'...  
4  [(add, , [('route_short_name', 'N55'), ('mode'...  
5  [(add, route, [(0, 'artificial_link===from:490...  
6  [(change, id, (new_service, more_appropriate_i...  
7  [(change, id, (new_route, more_appropriate_rou...

## Removing Stops, Routes, Services

In [27]:
n.schedule.remove_service('more_appropriate_id')

/Users/kasia.kozlowska/PycharmProjects/ABM/genet/genet/schedule_elements.py:1243: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  self.vehicles = {**df.T.to_dict(), **self.vehicles}
2021-10-25 11:29:00,295 - Removed Service with index `more_appropriate_id`, data={'id': 'more_appropriate_id', 'name': 'N55'} and Routes: {'more_appropriate_route_id'}


In [28]:
for route_id in {'another_new_route', 'another_new_route_2', 'another_new_route_3'}:
    n.schedule.remove_route(route_id)

2021-10-25 11:29:00,799 - Removed Route with index `another_new_route_3`, data={'route_short_name': 'N55', 'mode': 'bus', 'trips': {'trip_id': ['fun_trip_1', 'fun_trip_2'], 'trip_departure_time': ['03:53:00', '16:23:00'], 'vehicle_id': ['fun_bus_1', 'fun_bus_2']}, 'arrival_offsets': ['00:00:00', '00:02:00', '00:04:00', '00:06:00'], 'departure_offsets': ['00:00:00', '00:02:00', '00:04:00', '00:06:00'], 'route_long_name': '', 'id': 'another_new_route_3', 'route': [], 'await_departure': [True, True, True, True], 'ordered_stops': ['490000235X.link:834', 'new_stop', 'other_new_stop', '490010689KB.link:981']}. It was linked to Service `20274`.
2021-10-25 11:29:01,203 - Removed Route with index `another_new_route_2`, data={'route_short_name': 'N55', 'mode': 'bus', 'trips': {'trip_id': ['some_trip_1'], 'trip_departure_time': ['16:23:00'], 'vehicle_id': ['some_bus_2']}, 'arrival_offsets': ['00:00:00', '00:06:00'], 'departure_offsets': ['00:00:00', '00:06:00'], 'route_long_name': '', 'id': 'anot

In [29]:
n.schedule.change_log().tail()

timestamp change_event object_type               old_id  \
7   2021-10-25 11:28:59       modify       route            new_route   
8   2021-10-25 11:28:59       remove     service  more_appropriate_id   
9   2021-10-25 11:29:00       remove       route  another_new_route_3   
10  2021-10-25 11:29:00       remove       route  another_new_route_2   
11  2021-10-25 11:29:01       remove       route    another_new_route   

                       new_id  \
7   more_appropriate_route_id   
8                        None   
9                        None   
10                       None   
11                       None   

                                       old_attributes  \
7                                 {'id': 'new_route'}   
8        {'id': 'more_appropriate_id', 'name': 'N55'}   
9   {'route_short_name': 'N55', 'mode': 'bus', 'tr...   
10  {'route_short_name': 'N55', 'mode': 'bus', 'tr...   
11  {'route_short_name': 'N55', 'mode': 'bus', 'tr...   

                         new_attributes  \
7   {'id': 'more_appropriate_route_id'}   
8                                  None   
9                                  None   
10                                 None   
11                                 None   

                                                 diff  
7   [(change, id, (new_route, more_appropriate_rou...  
8   [(remove, , [('id', 'more_appropriate_id'), ('...  
9   [(remove, , [('route_short_name', 'N55'), ('mo...  
10  [(remove, , [('route_short_name', 'N55'), ('mo...  
11  [(remove, , [('route_short_name', 'N55'), ('mo...

You can also remove `Stop`s. This will disconnect `Route`s and `Service`s using that `Stop`s and likely render them invalid. The method will warn you which `Route`s and `Servce`s are affected.

In [30]:
n.schedule.remove_stop('new_stop')

2021-10-25 11:29:01,822 - Removed Stop with index `new_stop`, data={'id': 'new_stop', 'x': 529500.0, 'y': 181300.0, 'epsg': 'epsg:27700', 'name': 'New Stop', 'lon': -0.13507680987440782, 'lat': 51.51578819710294, 's2_id': 5221390696585158415}. Routes affected: {'20274_4'}. Services affected: {'20274'}.


You can also remove `Stop`s. This will disconnect `Route`s and `Service`s using that `Stop`s and likely render them invalid. The method will warn you which `Route`s and `Servce`s are affected.

In [31]:
n.schedule.remove_unsused_stops()

2021-10-25 11:29:01,885 - Removed Stop with index `490014214HE`, data={'id': '490014214HE', 'x': 529477.7501560802, 'y': 181314.43704307207, 'epsg': 'epsg:27700', 'name': 'Wardour Street (Stop OM)', 'lon': -0.13539198709361353, 'lat': 51.5159230332408, 's2_id': 5221390691069029927, 'isBlocking': 'false'}. Routes affected: set(). Services affected: set().
2021-10-25 11:29:01,897 - Removed Stop with index `490015042U`, data={'id': '490015042U', 'x': 529237.7588612114, 'y': 182408.0661155618, 'epsg': 'epsg:27700', 'name': 'Warren Street Stn  Euston Rd (Stop U)', 'lon': -0.13844798708907116, 'lat': 51.525806033239135, 's2_id': 5221390675812541383, 'isBlocking': 'false'}. Routes affected: set(). Services affected: set().
2021-10-25 11:29:01,914 - Removed Stop with index `490010531OS`, data={'id': '490010531OS', 'x': 528716.7250075269, 'y': 181154.11781636789, 'epsg': 'epsg:27700', 'name': 'New Bond Street (Stop OS)', 'lon': -0.1464119870992024, 'lat': 51.51465603324086, 's2_id': 52213665016

2021-10-25 11:29:02,166 - Removed Stop with index `490000252S`, data={'id': '490000252S', 'x': 529222.7900227709, 'y': 182423.4864806903, 'epsg': 'epsg:27700', 'name': 'Drummond Street (Stop S)', 'lon': -0.138657987089078, 'lat': 51.525948033239096, 's2_id': 5221390675795688803, 'isBlocking': 'false'}. Routes affected: set(). Services affected: set().
2021-10-25 11:29:02,174 - Removed Stop with index `490000078P`, data={'id': '490000078P', 'x': 529455.7452394223, 'y': 182401.37630677427, 'epsg': 'epsg:27700', 'name': 'Euston Square (Stop P)', 'lon': -0.13530998708775915, 'lat': 51.525696033239186, 's2_id': 5221390668020036699, 'isBlocking': 'false'}. Routes affected: set(). Services affected: set().
2021-10-25 11:29:02,186 - Removed Stop with index `490004695A`, data={'id': '490004695A', 'x': 529871.7640650324, 'y': 181148.23017527343, 'epsg': 'epsg:27700', 'name': 'Denmark Street (Stop A)', 'lon': -0.12977798709209976, 'lat': 51.514339033241114, 's2_id': 5221366120918541515, 'isBlocki

2021-10-25 11:29:02,370 - Removed Stop with index `490010689KB.link:981`, data={'id': '490010689KB.link:981', 'x': 529166.7349732723, 'y': 181256.33672284335, 'epsg': 'epsg:27700', 'name': 'Great Titchfield Street  Oxford Circus Station (Stop OP)', 'lon': -0.13989298709585662, 'lat': 51.51547203324081, 's2_id': 5221390688151572763, 'linkRefId': '981', 'isBlocking': 'false', 'stop_id': '490010689KB'}. Routes affected: set(). Services affected: set().
2021-10-25 11:29:02,381 - Removed Stop with index `490000091H`, data={'id': '490000091H', 'x': 528934.7945010976, 'y': 182202.18554864498, 'epsg': 'epsg:27700', 'name': 'Great Portland Street (Stop H)', 'lon': -0.14288798709207978, 'lat': 51.52402503323941, 's2_id': 5221390331888332895, 'isBlocking': 'false'}. Routes affected: set(). Services affected: set().
2021-10-25 11:29:02,407 - Removed Stop with index `9400ZZLUOXC4`, data={'id': '9400ZZLUOXC4', 'x': 529004.7147708246, 'y': 181277.2580580855, 'epsg': 'epsg:27700', 'name': 'Oxford Circ

2021-10-25 11:29:02,589 - Removed Stop with index `490000191A`, data={'id': '490000191A', 'x': 528649.7415935269, 'y': 182165.24841055647, 'epsg': 'epsg:27700', 'name': "Regent's Park (Stop A)", 'lon': -0.14700798709404783, 'lat': 51.52375803323939, 's2_id': 5221390302070799085, 'isBlocking': 'false'}. Routes affected: set(). Services affected: set().
2021-10-25 11:29:02,599 - Removed Stop with index `9400ZZLUWRR4`, data={'id': '9400ZZLUWRR4', 'x': 529225.7792770677, 'y': 182210.46930065827, 'epsg': 'epsg:27700', 'name': 'Warren Street Underground Station', 'lon': -0.13869298709023378, 'lat': 51.524033033239434, 's2_id': 5221390678156937061, 'stopAreaId': '940GZZLUWRR', 'isBlocking': 'false'}. Routes affected: set(). Services affected: set().
2021-10-25 11:29:02,607 - Removed Stop with index `490000235W1`, data={'id': '490000235W1', 'x': 529959.771919372, 'y': 181285.1243656829, 'epsg': 'epsg:27700', 'name': 'Tottenham Court Road Station (Stop S)', 'lon': -0.12845998709080084, 'lat': 5

## Modifying data stored for Stops, Routes, Services

### Applying known or pre-computed changes
Applying changes or new attributes to Services, Routes and Stops can be done via Schedule level methods. They all work with a dictionary where the keys are the object IDs and the values are dictionaries holding attribute names and values. The method to extract a DataFrame on attributes comes in handy here. E.g.

In [32]:
df = n.schedule.service_attribute_data(keys='name')
df.head()

name
20274  N55
12430  205
17732  N20
15660  113
14134   98

DataFrames are easy to work with. Youcould for exmaple manipulate the names or use other data to change these. For demonstration here, let's just set the names to something easy.

In [33]:
df['name'] = df['name'].apply(lambda x: f'Service_{x}')
df.head()

name
20274  Service_N55
12430  Service_205
17732  Service_N20
15660  Service_113
14134   Service_98

You can then convert this to a dictionary and pass it to the `apply_attributes_to_services` method.

In [34]:
n.schedule.apply_attributes_to_services(df.T.to_dict())

2021-10-25 11:29:02,733 - Changed Service attributes for 9 services


In [35]:
n.schedule.change_log().tail()

timestamp change_event object_type old_id new_id  \
92  2021-10-25 11:29:02       modify     service  14134  14134   
93  2021-10-25 11:29:02       modify     service  18853  18853   
94  2021-10-25 11:29:02       modify     service  18915  18915   
95  2021-10-25 11:29:02       modify     service  14073  14073   
96  2021-10-25 11:29:02       modify     service  15234  15234   

                    old_attributes                          new_attributes  \
92   {'id': '14134', 'name': '98'}   {'id': '14134', 'name': 'Service_98'}   
93   {'id': '18853', 'name': 'N8'}   {'id': '18853', 'name': 'Service_N8'}   
94   {'id': '18915', 'name': 'N5'}   {'id': '18915', 'name': 'Service_N5'}   
95   {'id': '14073', 'name': '94'}   {'id': '14073', 'name': 'Service_94'}   
96  {'id': '15234', 'name': '134'}  {'id': '15234', 'name': 'Service_134'}   

                                    diff  
92    [(change, name, (98, Service_98))]  
93    [(change, name, (N8, Service_N8))]  
94    [(change, name, (N5, Service_N5))]  
95    [(change, name, (94, Service_94))]  
96  [(change, name, (134, Service_134))]

You can do the same for `Routes` and `Stops`. Your dictionaries cannot however hold changes to indices. You will encounter an error and should use `reindex` methods for such operations.

In [36]:
n.schedule.apply_attributes_to_routes(
    {'VJ375a660d47a2aa570aa20a8568012da8497ffecf': {
        'name': 'my_favourite_route', 
        'mode': 'piggyback'
    }}
)

2021-10-25 11:29:02,807 - Changed Route attributes for 1 routes


In [37]:
n.schedule.apply_attributes_to_stops(
    {'490000235YB.link:574': {'new_attribute': 'hello!'}}
)

2021-10-25 11:29:02,823 - Changed Stop attributes for 1 stops


In [38]:
n.schedule.change_log().tail()

timestamp change_event object_type  \
94  2021-10-25 11:29:02       modify     service   
95  2021-10-25 11:29:02       modify     service   
96  2021-10-25 11:29:02       modify     service   
97  2021-10-25 11:29:02       modify       route   
98  2021-10-25 11:29:02       modify        stop   

                                        old_id  \
94                                       18915   
95                                       14073   
96                                       15234   
97  VJ375a660d47a2aa570aa20a8568012da8497ffecf   
98                        490000235YB.link:574   

                                        new_id  \
94                                       18915   
95                                       14073   
96                                       15234   
97  VJ375a660d47a2aa570aa20a8568012da8497ffecf   
98                        490000235YB.link:574   

                                       old_attributes  \
94                      {'id': '18915', 'name': 'N5'}   
95                      {'id': '14073', 'name': '94'}   
96                     {'id': '15234', 'name': '134'}   
97  {'route_short_name': 'N55', 'mode': 'bus', 'tr...   
98  {'services': {'20274', '18853', '14134'}, 'rou...   

                                       new_attributes  \
94              {'id': '18915', 'name': 'Service_N5'}   
95              {'id': '14073', 'name': 'Service_94'}   
96             {'id': '15234', 'name': 'Service_134'}   
97  {'route_short_name': 'N55', 'mode': 'piggyback...   
98  {'services': {'20274', '18853', '14134'}, 'rou...   

                                                 diff  
94                 [(change, name, (N5, Service_N5))]  
95                 [(change, name, (94, Service_94))]  
96               [(change, name, (134, Service_134))]  
97  [(change, mode, (bus, piggyback)), (add, , [('...  
98           [(add, , [('new_attribute', 'hello!')])]

You can use `route_trips_to_dataframe` to extract all of the trips, their departures and vehicle IDs associated with the trips in the schedule. Trip ids need not be unique, route IDs provide a secondary index. Associated service IDs are also given for convenience.

In [39]:
trips = n.schedule.route_trips_to_dataframe(gtfs_day='20210101')

In [40]:
trips.head()

route_id service_id  \
0  VJd78967364a302cf232c5139d40622dcb6c238c9e      14134   
1  VJd78967364a302cf232c5139d40622dcb6c238c9e      14134   
2  VJ235c8fca539cf931b3c673f9b056606384aff950      12430   
3  VJ235c8fca539cf931b3c673f9b056606384aff950      12430   
4  VJ235c8fca539cf931b3c673f9b056606384aff950      12430   

                                             trip_id trip_departure_time  \
0  VJ2750f36b3df447e3719dc10236b7daae59696f1d_23:... 2021-01-01 23:13:00   
1  VJd78967364a302cf232c5139d40622dcb6c238c9e_23:... 2021-01-01 23:05:00   
2  VJ02f66d6051cb3c787c2ec5430d41a33563c0be58_19:... 2021-01-01 19:30:30   
3  VJ0f837b7496df360a0ea669d199332f48c9c5cf88_19:... 2021-01-01 19:59:30   
4  VJ11da86a06c5be70ccf95b59df6f8c8b2c61a53a7_19:... 2021-01-01 19:05:30   

     vehicle_id  
0  veh_1144_bus  
1  veh_1145_bus  
2   veh_523_bus  
3   veh_524_bus  
4   veh_525_bus

Let's change all of the trip ids to something shorter

In [41]:
trips['trip_id'] = 'trip_' + trips.index.to_series().astype(str)
trips.head()

route_id service_id trip_id  \
0  VJd78967364a302cf232c5139d40622dcb6c238c9e      14134  trip_0   
1  VJd78967364a302cf232c5139d40622dcb6c238c9e      14134  trip_1   
2  VJ235c8fca539cf931b3c673f9b056606384aff950      12430  trip_2   
3  VJ235c8fca539cf931b3c673f9b056606384aff950      12430  trip_3   
4  VJ235c8fca539cf931b3c673f9b056606384aff950      12430  trip_4   

  trip_departure_time    vehicle_id  
0 2021-01-01 23:13:00  veh_1144_bus  
1 2021-01-01 23:05:00  veh_1145_bus  
2 2021-01-01 19:30:30   veh_523_bus  
3 2021-01-01 19:59:30   veh_524_bus  
4 2021-01-01 19:05:30   veh_525_bus

You can `set_route_trips_dataframe` which takes this dataframe and applies changes to all route trips based on the data in the dataframe. This means you can generate this DataFrame as shown below, manipulate trips (delete them, add new ones), change their departure times or change their vehicle ids to be shared for differnt trips, perhaps on some temporal logic and as long as the dataframe has the same schema, you can use it to set new trips in the schedule. This will appear in the changelog as a route level modify event.

Nb removing all trips of the same route from the dataframe will have no effect when being applied. If there is data in the dataframe for a route, all of its trips will be replaced by the data in the dataframe, and if there is no data for a route in the frame, no changes will be applied to that route (i.e. the trips attribute for routes missing from the dataframe will not be set as empty).

In [42]:
n.schedule.set_route_trips_dataframe(trips)
n.schedule.route_attribute_data(keys=[{'trips': 'trip_id'}])

2021-10-25 11:29:03,267 - Changed Route attributes for 69 routes


trips::trip_id
VJd78967364a302cf232c5139d40622dcb6c238c9e                                   [trip_0, trip_1]
VJ235c8fca539cf931b3c673f9b056606384aff950  [trip_2, trip_3, trip_4, trip_5, trip_6, trip_...
VJ1cf651142378958b52229bfe1fa552e49136e60e  [trip_32, trip_33, trip_34, trip_35, trip_36, ...
VJ652c769bc42361cc0308dff59a1fdcf0949bdade                                          [trip_78]
VJ9b58a59e3d74941586a5bca7726a8aa624da67fc  [trip_79, trip_80, trip_81, trip_82, trip_83, ...
...                                                                                       ...
VJb93a17a405fe502c5b3a2d6544105b0311da9fe2                             [trip_1531, trip_1532]
VJ8a4b1ca7dfd0a130abd1de9f55f3b756617dd4ca  [trip_1533, trip_1534, trip_1535, trip_1536, t...
VJe8cffad09738ff7b9698b333e3247918d5c45358  [trip_1541, trip_1542, trip_1543, trip_1544, t...
VJe18efadf172576fea7989ec1f233f26854c0f66a  [trip_1554, trip_1555, trip_1556, trip_1557, t...
VJ93d8207ae8540b4ff59d47c9ee1ec5689084522d                             [trip_1562, trip_1563]

[69 rows x 1 columns]

### Applying changes using functions or dictionary mappings

If you have some logic that can be written into a function of object's attributes, you can pass this to `apply_function_to_x` methods. You need to select `location`, which refers to the name of the attribute the result should be stored under. It can already exist and be overwritten. The function passed, is not expected to work with all objects. It will fail silently, only evaluating and generating outputs where possible.

In [43]:
from shapely.geometry import Point

def add_shapely_geometry_points(stop_attribs):
    return Point(stop_attribs['x'], stop_attribs['y'])

n.schedule.apply_function_to_stops(add_shapely_geometry_points, location='geometry')

2021-10-25 11:29:03,330 - Changed Stop attributes for 47 stops


In [44]:
n.schedule.change_log().tail(2)

timestamp change_event object_type          old_id  \
213  2021-10-25 11:29:03       modify        stop     490010689KB   
214  2021-10-25 11:29:03       modify        stop  other_new_stop   

             new_id                                     old_attributes  \
213     490010689KB  {'id': '490010689KB', 'x': 529166.7349732723, ...   
214  other_new_stop  {'services': {'20274'}, 'routes': {'20274_4'},...   

                                        new_attributes  \
213  {'id': '490010689KB', 'x': 529166.7349732723, ...   
214  {'services': {'20274'}, 'routes': {'20274_4'},...   

                                                  diff  
213  [(add, , [('geometry', <shapely.geometry.point...  
214  [(add, , [('geometry', <shapely.geometry.point...

In [45]:
n.schedule.stop_attribute_data(keys=['name', 'x', 'y', 'geometry']).head()

name  \
490000235X.link:834                Tottenham Court Road Station (Stop X)   
490000235YB.link:574                Oxford Street  Soho Street (Stop YB)   
490014214HE.link:3154                           Wardour Street (Stop OM)   
490000235P.link:15     Tottenham Court Road Station  New Oxford Stree...   
490015196OG.link:2954                            Holles Street (Stop OG)   

                                   x              y  \
490000235X.link:834    529981.795880  181412.097576   
490000235YB.link:574   529570.781323  181336.281593   
490014214HE.link:3154  529477.750156  181314.437043   
490000235P.link:15     529967.770031  181420.307095   
490015196OG.link:2954  528823.776634  181291.362193   

                                                          geometry  
490000235X.link:834    POINT (529981.7958802709 181412.0975758662)  
490000235YB.link:574   POINT (529570.7813227688 181336.2815925331)  
490014214HE.link:3154  POINT (529477.7501560802 181314.4370430721)  
490000235P.link:15      POINT (529967.770031479 181420.3070953883)  
490015196OG.link:2954   POINT (528823.776634044 181291.3621925522)

In [46]:
from geopandas import GeoDataFrame
GeoDataFrame(n.schedule.stop_attribute_data(keys='geometry')).plot()

In [47]:
n.schedule.stop('490000235YB.link:574').__dict__

{'id': '490000235YB.link:574',
 'x': 529570.7813227688,
 'y': 181336.2815925331,
 'epsg': 'epsg:27700',
 'name': 'Oxford Street  Soho Street (Stop YB)',
 'lat': 51.51609803324077,
 'lon': -0.13404398709291904,
 's2_id': 5221390696959560815,
 'linkRefId': '574',
 'isBlocking': 'false',
 'new_attribute': 'hello!',
 'geometry': <shapely.geometry.point.Point at 0x125741a10>}

For this let's say we want to reduce the number of trips. For simplicity of demonstration we don't have about which trips we delete, but logic around timings of trips can be added in this function, as the trips are saved as one of routes attributes (check out the summary methods)

In [48]:
# before
len(n.schedule.route_trips_to_dataframe())

1564

In [49]:
def reduce_trips(attribs):
    # just delete any other trip
    attribs['trips']['trip_id'] = attribs['trips']['trip_id'][::2]
    attribs['trips']['trip_departure_time'] = attribs['trips']['trip_departure_time'][::2]
    attribs['trips']['vehicle_id'] = attribs['trips']['vehicle_id'][::2]
    return attribs['trips']

n.schedule.apply_function_to_routes(reduce_trips, 'trips')

2021-10-25 11:29:04,202 - Changed Route attributes for 69 routes


In [50]:
# after
len(n.schedule.route_trips_to_dataframe())

793

Note, this could also be done using the `route_trips_to_dataframe` and `set_route_trips_dataframe` mentioned above.

Let's give an example of using a mapping. We can re-use the service name DataFrame we generated above.

In [51]:
df['new_name'] = 'Brand_new_name' + df['name']
df.head()

name                   new_name
20274  Service_N55  Brand_new_nameService_N55
12430  Service_205  Brand_new_nameService_205
17732  Service_N20  Brand_new_nameService_N20
15660  Service_113  Brand_new_nameService_113
14134   Service_98   Brand_new_nameService_98

In [52]:
name_map = dict(zip(df['name'], df['new_name']))
name_map

{'Service_N55': 'Brand_new_nameService_N55',
 'Service_205': 'Brand_new_nameService_205',
 'Service_N20': 'Brand_new_nameService_N20',
 'Service_113': 'Brand_new_nameService_113',
 'Service_98': 'Brand_new_nameService_98',
 'Service_N8': 'Brand_new_nameService_N8',
 'Service_N5': 'Brand_new_nameService_N5',
 'Service_94': 'Brand_new_nameService_94',
 'Service_134': 'Brand_new_nameService_134'}

In this case, `location` refers to the attribute to be mapped.

In [53]:
n.schedule.apply_function_to_services(name_map, location='name')

2021-10-25 11:29:04,359 - Changed Service attributes for 9 services


In [54]:
n.schedule.change_log().tail()

timestamp change_event object_type old_id new_id  \
288  2021-10-25 11:29:04       modify     service  14134  14134   
289  2021-10-25 11:29:04       modify     service  18853  18853   
290  2021-10-25 11:29:04       modify     service  18915  18915   
291  2021-10-25 11:29:04       modify     service  14073  14073   
292  2021-10-25 11:29:04       modify     service  15234  15234   

                             old_attributes  \
288   {'id': '14134', 'name': 'Service_98'}   
289   {'id': '18853', 'name': 'Service_N8'}   
290   {'id': '18915', 'name': 'Service_N5'}   
291   {'id': '14073', 'name': 'Service_94'}   
292  {'id': '15234', 'name': 'Service_134'}   

                                        new_attributes  \
288  {'id': '14134', 'name': 'Brand_new_nameService...   
289  {'id': '18853', 'name': 'Brand_new_nameService...   
290  {'id': '18915', 'name': 'Brand_new_nameService...   
291  {'id': '14073', 'name': 'Brand_new_nameService...   
292  {'id': '15234', 'name': 'Brand_new_nameService...   

                                                  diff  
288  [(change, name, (Service_98, Brand_new_nameSer...  
289  [(change, name, (Service_N8, Brand_new_nameSer...  
290  [(change, name, (Service_N5, Brand_new_nameSer...  
291  [(change, name, (Service_94, Brand_new_nameSer...  
292  [(change, name, (Service_134, Brand_new_nameSe...